In [ ]:
import os
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [47]:
dir = 'russian_corpus'
filepaths = []

for i in os.listdir(dir):
    filepaths.append(os.path.join(dir, i))  

corpus = []
tokens = []

for filepath in filepaths:
    if 'Война и мир' in filepath:
        with open(filepath, encoding='utf-8') as f:
            text = f.read()
            text.replace('.', '')
            text = [x for x in word_tokenize(text) if x.isalpha()]
            if text:
                tokens.extend(text)
                text = [x for x in [text[0]] if len(x) > 3] + [x for x in text[1:] if not x.istitle() and len(x) > 3]
                text = ' '.join(text)
                corpus.append(text)

print(f'Количество словоформ: {len(tokens)}')
print(f'Количество токенов: {len(set(tokens))}')
print(f'Количество документов в корпусе: {len(corpus)}')

with open('ru_stop_words.txt', encoding='utf-8') as f:
    STOPWORDS = f.read().split()
    print(STOPWORDS)

Количество словоформ: 113260
Количество токенов: 20330
Количество документов в корпусе: 10896
['a', 'about', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', 'could', 'do', 'for', 'from', 'has', 'have', 'i', 'if', 'in', 'is', 'it', 'me', 'my', 'no', 'not', 'of', 'on', 'one', 'or', 'so', 'that', 'the', 'them', 'there', 'they', 'this', 'to', 'was', 'we', 'what', 'which', 'will', 'with', 'would', 'you', 'а', 'будем', 'будет', 'будете', 'будешь', 'буду', 'будут', 'будучи', 'будь', 'будьте', 'бы', 'был', 'была', 'были', 'было', 'быть', 'в', 'вам', 'вами', 'вас', 'весь', 'во', 'вот', 'все', 'всё', 'всего', 'всей', 'всем', 'всём', 'всеми', 'всему', 'всех', 'всею', 'всея', 'всю', 'вся', 'вы', 'да', 'для', 'до', 'его', 'едим', 'едят', 'ее', 'её', 'ей', 'ел', 'ела', 'ем', 'ему', 'емъ', 'если', 'ест', 'есть', 'ешь', 'еще', 'ещё', 'ею', 'же', 'за', 'и', 'из', 'или', 'им', 'ими', 'имъ', 'их', 'к', 'как', 'кем', 'ко', 'когда', 'кого', 'ком', 'кому', 'комья', 'кот

In [16]:
import random

for i in range(100):
    # print(random.choice(tokens))

SyntaxError: unexpected EOF while parsing (<ipython-input-16-3334b4417016>, line 5)

In [51]:
no_features = 20000
no_topics = 20

tfidf_vectorizer = TfidfVectorizer(
    max_df=0.7,
    # min_df=3, 
    stop_words=STOPWORDS, 
    max_features=no_features
)
data_tfidf_vectorized = tfidf_vectorizer.fit_transform(corpus)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()


nmf = NMF(n_components=no_topics, 
          random_state=1, alpha=.01, l1_ratio=.5, init='nndsvd'
          )
nmf_tfidf_model = nmf.fit(data_tfidf_vectorized)
print(nmf_tfidf_model.transform(data_tfidf_vectorized).shape)

(10896, 20)


In [5]:
def print_topics(model, feature_names, top_n=10):
    for topic_idx, topic in enumerate(model.components_):
            print("Topic %d:" % (topic_idx))
            print(" ".join([feature_names[i] for i in topic.argsort()[:-top_n - 1:-1]]))

In [52]:
print_topics(nmf_tfidf_model, tfidf_feature_names)


Topic 0:
князь старый повторил перебил думаете тихо вышел князю спрашивал тоном
Topic 1:
княжна милая père давно сказали головой звала пойду отвечая комнаты
Topic 2:
ваше сиятельство благородие превосходительство величество прикажете высокопревосходительство пожалуйте лёгко готов
Topic 3:
граф старый закричал милый молодой думаете chère любит вопросительно задумался
Topic 4:
зачем взял сюда поедете привел графа понять ездит оттого пришла
Topic 5:
делать нечего аккуратно спать скажите трудно человеком умеете народом отказаться
Topic 6:
друг друга любезный время милый право думаешь знал сердечный видишь
Topic 7:
куда пожалуйте прикажете положил рано поедет богу привязать делся фатает
Topic 8:
графиня поди давно пойду сюда шопотом спрашивала дочь мучит удивлением
Topic 9:
государь генерал император нахмурившись милостивый изволил говорили милостиво масон более
Topic 10:
крикнул офицер другой кричал нахмурившись голосом пошел повторил время сердито
Topic 11:
разве равно думать немцы слышиш